In [ ]:
!pip install apache_beam

In [ ]:
import apache_beam as beam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/Cloud-AI-Analytics/Apache\ Beam\ -Python/data

In [ ]:
!ls

In [ ]:
!{('head -n 10 cloud_export_100.json')}

## **Partition**:

•	Partition is a Beam transform for PCollection objects that store the same data type. It splits a single PCollection into a fixed number of smaller collections.

•	Partition divides the elements of a PCollection according to a partitioning function that you provide.

•	The partitioning function contains the logic that determines how to split up the elements of the input PCollection into each resulting partition PCollection.

•	The number of partitions must be determined at graph construction time.

•	Partition accepts a function that receives the number of partitions, and returns the index of the desired partition for the element. The number of partitions passed must be a positive integer, and it must return an integer in the range 0 to num_partitions-1.



In [ ]:
services = ['Compute Engine', 'Networking', 'Cloud Logging', 'Support']

def by_service(cloud, num_partitions):
  return services.index(cloud['service_description'])

with beam.Pipeline() as pipeline:
  computes, networks, logs, supports = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          {"account_id":"01097B","service_id":"6F81-5844-456A","service_description":"Compute Engine","cost":"0.000372","usage_start_time":"2024-08-26 00:00:00.000000 UTC","usage_end_time":"2024-08-26 01:00:00.000000 UTC","export_time":"2024-08-26 06:15:49.620818 UTC"},
          {"account_id":"01097B","service_id":"E505-1604-58F8","service_description":"Networking","cost":"0.0","usage_start_time":"2024-08-27 15:00:00.000000 UTC","usage_end_time":"2024-08-27 16:00:00.000000 UTC","export_time":"2024-08-27 20:31:26.191136 UTC"},
          {"account_id":"01097B","service_id":"2062-016F-44A2","service_description":"Support","cost":"0.011923","usage_start_time":"2024-08-06 09:00:00.000000 UTC","usage_end_time":"2024-08-06 10:00:00.000000 UTC","export_time":"2024-08-06 17:35:05.487018 UTC"},
          {"account_id":"01097B","service_id":"5490-F7B7-8DF6","service_description":"Cloud Logging","cost":"-1e-06","usage_start_time":"2024-08-15 08:00:00.000000 UTC","usage_end_time":"2024-08-15 09:00:00.000000 UTC","export_time":"2024-08-15 17:21:06.500508 UTC"},
      ])
      | 'Partition' >> beam.Partition(by_service, len(services))
  )

  computes  | 'Computes' >> beam.Map(
          lambda x: print('Compute Engine: {}'.format(x)))
  networks  | 'Networks' >> beam.Map(
          lambda x: print('Networking: {}'.format(x)))
  logs    | 'Logs' >> beam.Map(
          lambda x: print('Cloud Logging: {}'.format(x)))
  supports  | 'Supports' >> beam.Map(
          lambda x: print('Support: {}'.format(x)))


將上面程式碼 beam.Create 改為 讀取JSON文件檔，名稱為 cloud_export_100.json

In [ ]:
# prompt: 將上面程式碼 beam.Create 改為 讀取JSON文件檔，名稱為 cloud_export_100.json

import apache_beam as beam
!pip install apache_beam
# ## **Partition**:
#
# •	Partition is a Beam transform for PCollection objects that store the same data type. It splits a single PCollection into a fixed number of smaller collections.
#
# •	Partition divides the elements of a PCollection according to a partitioning function that you provide.
#
# •	The partitioning function contains the logic that determines how to split up the elements of the input PCollection into each resulting partition PCollection.
#
# •	The number of partitions must be determined at graph construction time.
#
# •	Partition accepts a function that receives the number of partitions, and returns the index of the desired partition for the element. The number of partitions passed must be a positive integer, and it must return an integer in the range 0 to num_partitions-1.
#
#
services = ['Compute Engine', 'Networking', 'Cloud Logging', 'Support']

def by_service(cloud, num_partitions):
  return services.index(cloud['service_description'])

with beam.Pipeline() as pipeline:
  computes, networks, logs, supports = (
      pipeline
      | 'Read from JSON' >> beam.io.ReadFromText('cloud_export_100.json')
      | 'Parse JSON' >> beam.Map(eval)  # Assuming each line is a valid JSON object
      | 'Partition' >> beam.Partition(by_service, len(services))
  )

  computes  | 'Computes' >> beam.Map(
          lambda x: print('Compute Engine: {}'.format(x)))
  networks  | 'Networks' >> beam.Map(
          lambda x: print('Networking: {}'.format(x)))
  logs    | 'Logs' >> beam.Map(
          lambda x: print('Cloud Logging: {}'.format(x)))
  supports  | 'Supports' >> beam.Map(
          lambda x: print('Support: {}'.format(x)))

## **Flatten**:

•	Flatten is a Beam transform for PCollection objects that store the same data type. Flatten merges multiple PCollection objects into a single logical PCollection.

•	Kind of Union operation


In [ ]:
# prompt: 將上面 beam.Partition()的集合，使用 beam.Flatten() 來合併，然後印出來

services = ['Compute Engine', 'Networking', 'Cloud Logging', 'Support']

def by_service(cloud, num_partitions):
  return services.index(cloud['service_description'])

with beam.Pipeline() as pipeline:
  computes, networks, logs, supports = (
      pipeline
      | 'Read from JSON' >> beam.io.ReadFromText('cloud_export_100.json')
      | 'Parse JSON' >> beam.Map(eval)  # Assuming each line is a valid JSON object
      | 'Partition' >> beam.Partition(by_service, len(services))
  )

  merged_pcollection = (
      (computes, networks, logs, supports)
      | 'Flatten PCollections' >> beam.Flatten()
      | 'Print Merged' >> beam.Map(print)
  )